In [1]:
import sys, os, importlib, multiprocessing
import rasterio, dask

import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import GOSTRocks.rasterMisc as rMisc

from GOSTRocks.misc import tPrint
from dask.distributed import Client, LocalCluster
from shapely.geometry import Point, LineString

sys.path.append("../src")

import GOST_AIS.ais_data_helper as ais

In [2]:
n_workers = 32 # multiprocessing.cpu_count()
threads_per_worker = 4

try:
    client = Client('127.0.0.1:8786')
except:
    cluster = LocalCluster(n_workers = n_workers, processes=False, threads_per_worker=threads_per_worker, scheduler_port=8786)
    client = Client(cluster)
    
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 128,Total memory: 0.98 TiB
Status: running,Using processes: False
Comm: tcp://127.0.0.1:8786,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:41311,Total threads: 4
Dashboard: http://127.0.0.1:42757/status,Memory: 31.49 GiB
Nanny: None,


In [3]:
ais_folder = '/home/jupyter-wb411133/data/global/AIS'
sample_file = os.path.join(ais_folder, "WorldBank_SAIS_globalAOI_20190101_20201231_000000000001.csv")
inD = pd.read_csv(sample_file)
inD.head()

/home/jupyter-wb411133/.conda/envs/geog/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,12,13,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,created_at,timestamp,mmsi,msg_type,latitude,longitude,speed,course,heading,rot,...,eta,destination,status,maneuver,accuracy,collection_type,to_bow,to_stern,to_port,to_starboard
0,2019-05-27 11:25:46.377134 UTC,2019-05-27 07:35:39 UTC,247355400,1,-29.578107,31.598613,14.2,236.5,238.0,-18.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN
1,2020-12-12 23:36:28.711465 UTC,2020-12-12 23:17:11 UTC,645157000,1,-6.423307,53.156427,6.9,63.1,64.0,13.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN
2,2020-10-22 14:39:52.651258 UTC,2020-10-22 13:29:44 UTC,226327000,3,47.359573,-154.588933,12.7,294.0,282.0,14.0,...,NaN,NaN,0.0,0.0,1.0,satellite,NaN,NaN,NaN,NaN
3,2019-10-20 11:17:19.062695 UTC,2019-10-20 08:03:03 UTC,578000500,1,-9.965413,-139.126000,0.2,335.9,2.0,11.0,...,NaN,NaN,0.0,0.0,1.0,satellite,NaN,NaN,NaN,NaN
4,2020-01-19 03:21:11.402599 UTC,2020-01-18 23:11:43 UTC,238311000,3,13.714533,176.218160,13.2,280.1,281.0,11.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN


In [4]:
col_types = {'created_at': str,
                         'timestamp': str,
                         'mmsi': int,
                         'msg_type': int,
                         'latitude': float,
                         'longitude': float,
                         'speed': float,
                         'course': float,
                         'heading': float,
                         'rot': float,
                         'imo': float,
                         'name': str,
                         'call_sign': str,
                         'flag': str,
                         'draught': float,
                         'ship_and_cargo_type': str, 
                         'length': float,
                         'width': float,
                         'eta': str,
                         'destination': str,
                         'status': float,
                         'maneuver': float,
                         'accuracy': float,
                         'collection_type': str,
                         'to_bow': float,
                         'to_stern': float,
                         'to_port': float,
                         'to_starboard': float}


# Extract data by geography

In [8]:
# Generate two queries to select the Eastern and Western halves of the data around the international date line
adm0 = gpd.read_file('/home/jupyter-wb411133/data/regional/pacific_observatory/admin/Adm0_Pacific.shp')
adm0 = adm0.to_crs('epsg:4326')
adm0['min_lon'] = adm0['geometry'].apply(lambda x: x.bounds[0])
adm0['min_lat'] = adm0['geometry'].apply(lambda x: x.bounds[1])
adm0['max_lon'] = adm0['geometry'].apply(lambda x: x.bounds[2])
adm0['max_lat'] = adm0['geometry'].apply(lambda x: x.bounds[3])

# As the set of countries cross the international date line, we need to do some math to 
#    determine the actual longitude bounds
westing = adm0.loc[adm0['min_lon'] > 0]['min_lon'].min()
easting = adm0.loc[adm0['max_lon'] < 0]['max_lon'].max()
southing = adm0.total_bounds[1]
northing = adm0.total_bounds[3]

print(f'{westing} : {easting}, {southing} : {northing}')

131.16923522868194 : -124.7728500366018, -29.119367598677027 : 28.398052216186542


In [9]:
inD = dd.read_csv('/home/jupyter-wb411133/data/global/AIS/WorldBank_SAIS_globalAOI_20190101_20201231_0000000031**.csv', dtype=col_types)

In [10]:
inD.head()

,created_at,timestamp,mmsi,msg_type,latitude,longitude,speed,course,heading,rot,...,eta,destination,status,maneuver,accuracy,collection_type,to_bow,to_stern,to_port,to_starboard
0,2019-12-25 01:02:06.360786 UTC,2019-12-24 18:54:33 UTC,345140011,1,19.078987,-104.291840,0.0,340.7,232.0,2.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN
1,2019-08-30 06:12:45.651906 UTC,2019-08-30 04:24:59 UTC,345010045,1,19.423013,-91.984053,8.7,174.0,168.0,12.0,...,NaN,NaN,0.0,0.0,1.0,satellite,NaN,NaN,NaN,NaN
2,2019-06-20 01:22:30.302239 UTC,2019-06-19 23:42:36 UTC,310515000,1,4.842747,-46.351867,11.8,307.1,313.0,12.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN
3,2020-06-05 17:06:05.435762 UTC,2020-06-05 15:34:53 UTC,247330800,1,-4.523893,11.415093,1.9,261.0,129.0,-13.0,...,NaN,NaN,8.0,0.0,1.0,satellite,NaN,NaN,NaN,NaN
4,2020-06-27 16:26:09.049252 UTC,2020-06-27 16:03:24 UTC,345080900,1,20.106480,-116.113627,12.0,3.6,0.0,-17.0,...,NaN,NaN,0.0,0.0,0.0,satellite,NaN,NaN,NaN,NaN


In [ ]:
# Subset to just fishing boats
inD = inD.loc[inD['ship_and_cargo_type'] == '30']

#Now, do two select by locations 
tPrint("Starting")
eastern_hemi = inD.loc[(inD['longitude'] > westing) & (inD['latitude'] > southing) & (inD['latitude'] < northing)].compute()
tPrint("Eastern Done")
western_hemi = inD.loc[(inD['longitude'] < easting) & (inD['latitude'] > southing) & (inD['latitude'] < northing)].compute()
tPrint("Western Done")

14:51:40	Starting


distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 25.38 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 25.56 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 25.84 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 26.27 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 85% memory usage. Pausing worker.  Process memory: 26.89 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 86% memory usage. Pausing worker.  Process memory: 27.10 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNING - Worker is at 87% memory usage. Pausing worker.  Process memory: 27.50 GiB -- Worker memory limit: 31.49 GiB
distributed.worker - WARNIN

# Generate routes per boat

In [ ]:
importlib.reload(ais)
out_folder = '/home/jupyter-wb411133/data/global/AIS_output'
    
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

def create_ship_tracks(curD):
    try:
        mmsi = curD['mmsi'][0]
        out_folder = '/home/jupyter-wb411133/data/global/AIS_output'
        curAIS = ais.ais_helper(inD = curD)
        curShp = curAIS.read_simple_geom()
        lines = ais.generate_linear_features(curShp)
        lines['mmsi'] = mmsi
        out_file = os.path.join(out_folder, f"{mmsi}.geojson")
        try:
            lines.to_file(out_file, driver="GeoJSON")
        except:
            pass
        return(lines)
    except:
        pass
    
res = inD.groupby("mmsi").apply(create_ship_tracks).compute()

In [ ]:
inD.groupby("mmsi").count().compute()

In [ ]:
importlib.reload(ais)
curD = inD.loc[inD['mmsi'] == unique_boats[0]]
curD.head()

In [ ]:
importlib.reload(ais)


In [ ]:
importlib.reload(ais)

in_file = 's3://wbgdecinternal-ntl/AIS_TMP/MMSI_247298800.csv'
inD = ais.ais_file(in_file)
tempD = inD.read_simple_geom()
tempD['DAY'] = tempD['timestamp'].apply(lambda x: x[:10])

In [ ]:
tempD.to_file(f'/home/wb411133/temp/AIS/{os.path.basename(in_file).replace(".csv","")}.geojson', driver="GeoJSON")

In [ ]:
# Generate lines file for individual ship data
importlib.reload(ais)
line_features = ais.generate_linear_features(tempD)
line_features.to_file(f'/home/wb411133/temp/AIS/{os.path.basename(in_file).replace(".csv","_lines_simple")}.geojson', driver="GeoJSON")

# Processing daily files

In [ ]:
daily_file = 's3://wbgdecinternal-ntl/AIS_TMP/MAY2.csv'
inD = pd.read_csv(daily_file)

In [ ]:
inD = inD.loc[~inD['latitude'].isna()]
inD['DAY'] = inD['timestamp'].apply(lambda x: x[:10])

In [ ]:
inD['DAY'].value_counts()

In [ ]:
inD_geom = [Point(x['longitude'], x['latitude']) for idx, x in inD.iterrows()]
inGPD = gpd.GeoDataFrame(inD, geometry=inD_geom, crs='epsg:4326')

In [ ]:
rMisc.rasterizeDataFrame?

In [ ]:
out_file = "/home/wb411133/temp/AIS/may2.tif"
rMisc.rasterizeDataFrame(inGPD, out_file, res=0.1, mergeAlg="ADD")